In [1]:
%matplotlib inline

import numpy as np;
import numba as jit;
import matplotlib.pyplot as plt
from IPython import display

In [3]:
# GRID

#             5    6
#
#           4    0    1
#
#             3    2

# Define variables used
relaxation_constant      = 1E8;
grid_step_x              = 1E-1;
grid_step_y              = 1E-1;
speed_of_sound           = 1;
time_step                = 1 / speed_of_sound;
directional_weights      = np.array([1/2, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12]);
unit_x_velocities        = np.array([
        0,
        1,
        1/2,
        -1/2,
        -1,
        -1/2,
        1/2]); # Rotating counter-clockwise
unit_y_velocities        = np.array([
        0,
        0,
        np.sqrt(3)/2,
        np.sqrt(3)/2,
        0,
        -np.sqrt(3)/2,
        -np.sqrt(3)/2])
ei                       = np.stack((unit_x_velocities, unit_y_velocities));
density                  = 1E0;
mass                     = 1E0;
particles_per_grid_point = 1E1;
average_density          = particles_per_grid_point;
even                     = np.int_([[0,0],[1,0],[1,1],[0,1],[-1,0],[0,-1],[1,-1]]);
odd                      = np.int_([[0,0],[1,0],[0,1],[-1,1],[-1,0],[-1,-1],[0,-1]]);

# Create the grid. We use a triangular grid
x          = np.linspace(0, 1, 1 / grid_step_x + 1);
y          = np.linspace(-grid_step_y/2, 1 + grid_step_y/2, 1/grid_step_y + 2);
[X, Y]     = np.meshgrid(x,y);                     # This is a rectangular grid
X[1::2,:]  = X[0::2,:] + (X[1,1] - X[2,1] / 2);    # offset every second row to make it triangular

# Create the velocity grid.
u           = np.ones(np.shape(x));                 # Initial x velocity is uniformly 1
v           = np.zeros(np.shape(y));
[U, V]      = np.meshgrid(u,v);
# U[[0,-1],:] = 0;                                    # Set the x-velocity 0 at the boundaries

# Create the particles
fi          = np.zeros((X.shape[0], X.shape[1], 7))
print(fi.shape);

(12, 11, 7)


In [7]:
# Next we are going to run the simulation
time_steps = np.int(np.ceil(1 / time_step * 300)+1);   # Simulate 1 second
#Particle_Locations = np.zeros(np.append(np.shape(Particles), time_steps));

time_steps = 3;

C = speed_of_sound;
tau = relaxation_constant;

for t in range(1, time_steps):
    #Calculate the new fi
    rfi  = np.zeros(np.shape(fi));
    fieq = np.zeros(np.shape(fi));
        
    rfi = fi;
    rfi[2,1,0] = 999;
    
    fi = np.zeros(fi.shape);
    rfio = rfi; rfie = rfi;
    rfie[0::2,:,:] = 0;
    rfio[1::2,:,:] = 0;
    for i in range(0,7):
        print(rfi[:,:,i]);
        print(rfie[:,:,i])
        print(rfio[:,:,i])
        continue;
        #odd
        rfioa = np.roll(
            np.roll(
                np.squeeze(
                    rfio[:,:,i]
                ), np.int(odd[i,0]),axis=0
            ), np.int(odd[i,0]), axis=1
        );
        print(rfioa);
        fi[:,:,i] = fi[:,:,i] + rfioa;
        #even
        rfiea = np.roll(
            np.roll(
                np.squeeze(
                    rfio[:,:,i]
                ), np.int(even[i,0]),axis=0
            ), np.int(even[i,1]), axis=1
        );
        print(rfiea);
        fi[:,:,i] = fi[:,:,i] + rfiea;
        #print(fi);
    
        
    # Boundary conditions
    #for i in range(7):
    #fi[-1,:,:] = fi[-2,:,:];
    #for i in range(7):
        #fi[0,:,i] = -fi[1,:,i];
        #fi[-1,:,i] = -fi[-2,:,i];
    #fi[0,:,:] = -fi[1,:,:];
    #fi[0,:,1:3] = 0;
    #fi[-1,:,:] = -fi[-2,:,:];
    #fi[-1,:,4:6] = 0;
    
    
    rho = np.sum(fi, axis=2);
    print(rho);
    print(np.sum(rho));
    temp1 = np.multiply(fi,ei[0,:]);
    temp1 = np.sum(temp1, axis=2);
    temp2 = np.multiply(fi,ei[1,:]);
    temp2 = np.sum(temp2, axis=2);
    U = np.divide(temp1, rho);
    V = np.divide(temp2, rho);
    U = np.nan_to_num(U);
    V = np.nan_to_num(V);
    
    # Apply boundary conditions
    #U[:,0] = 1E-3;
    #U[6,0] = 0;
    continue;
    if (t%10==0):
        fig = plt.figure();
        Q = plt.quiver(X,Y,U,V);
        #Q = plt.quiver(X[::1,::1],Y[::1,::10],U[::1,::10],V[::1,::10],units='width');
        qk = plt.quiverkey(Q, 0.9, 0.95, 2, r'$2 \frac{m}{s}$',
                   labelpos='E',
                   coordinates='figure',
                   fontproperties={'weight': 'bold'})
        display.clear_output(wait=True);
        display.display(fig);
        print(t);

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0

C:\Users\sybre_000\Anaconda3\lib\site-packages\ipykernel\__main__.py:69: RuntimeWarning: invalid value encountered in true_divide
C:\Users\sybre_000\Anaconda3\lib\site-packages\ipykernel\__main__.py:70: RuntimeWarning: invalid value encountered in true_divide
